<a href="https://colab.research.google.com/github/curoled345/Automoblie-Project/blob/main/0709_OpencCV_MOG2_%EC%B0%A8%EB%9F%89%EA%B0%90%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 기본 MOG2 차량 감지 코드

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt


# 동영상 파일 열기 (코랩에서는 업로드한 파일 경로 사용)
cap = cv2.VideoCapture('/content/sample_data/around5.mp4')  # 파일 경로 수정 필요


# MOG2 배경 차분기 생성
backSub = cv2.createBackgroundSubtractorMOG2()


frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 배경 차분 적용
    fgMask = backSub.apply(frame)

    # 매 30프레임마다 결과 출력 (너무 많은 출력 방지)
    if frame_count % 30 == 0:   # 30초마다 1프레임
        # 결과를 나란히 표시
        combined = np.hstack((frame, cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 정도만 처리 (테스트용)
    if frame_count > 100:
        break


# 자원 해제
cap.release()
print("처리 완료!")


# + 노이즈 제거 (모폴로지 연산)

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt


# 동영상 파일 열기 (코랩에서는 업로드한 파일 경로 사용)
cap = cv2.VideoCapture('/content/sample_data/around5.mp4')  # 파일 경로 수정 필요


# MOG2 배경 차분기 생성
backSub = cv2.createBackgroundSubtractorMOG2()


frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 배경 차분 적용
    fgMask = backSub.apply(frame)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)  # 작은 노이즈 제거
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)  # 구멍 메우기


    # 매 30프레임마다 결과 출력 (너무 많은 출력 방지)
    if frame_count % 30 == 0:   # 30초마다 1프레임
        # 결과를 나란히 표시
        combined = np.hstack((frame, cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 정도만 처리 (테스트용)
    if frame_count > 100:
        break


# 자원 해제
cap.release()
print("처리 완료!")

# + 윤곽선 검출 및 바운딩 박스

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt


# 동영상 파일 열기 (코랩에서는 업로드한 파일 경로 사용)
cap = cv2.VideoCapture('/content/sample_data/around5.mp4')  # 파일 경로 수정 필요


# MOG2 배경 차분기 생성
backSub = cv2.createBackgroundSubtractorMOG2()


frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 배경 차분 적용
    fgMask = backSub.apply(frame)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)  # 작은 노이즈 제거
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)  # 구멍 메우기

    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 바운딩 박스를 그릴 프레임 복사
    result_frame = frame.copy()

    for contour in contours:
        # 너무 작은 영역은 제외 (차량이 아닐 가능성 높음)
        if cv2.contourArea(contour) > 3000:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(result_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)



    # 매 30프레임마다 결과 출력 (너무 많은 출력 방지)
    if frame_count % 30 == 0:   # 30초마다 1프레임
        # 결과를 나란히 표시
        combined = np.hstack((frame, cv2.cvtColor(fgMask, cv2.COLOR_GRAY2BGR)))
        cv2_imshow(combined)

    frame_count += 1

    # 100프레임 정도만 처리 (테스트용)
    if frame_count > 100:
        break


# 자원 해제
cap.release()
print("처리 완료!")

# - 최종본

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# 동영상 파일 열기
cap = cv2.VideoCapture('/content/sample_data/around5.mp4')

# ROI 생성 함수
def create_lane_roi(frame):
    height, width = frame.shape[:2]
    mask = np.zeros((height, width), dtype=np.uint8)

    # ▶ 오른쪽으로 조금 더 이동하고 밑변을 오른쪽으로 넓히고 왼쪽 밑변을 줄이고 왼쪽 윗변을 줄인 사다리꼴 ROI
    # 모든 x 좌표에 오프셋 추가하여 오른쪽으로 이동
    offset_x = int(width * 0.01) # 오른쪽으로 이동할 픽셀 수 (대략 1cm에 해당하도록 조정)

    roi_points = np.array([
        [int(width * 0.2) + offset_x, height],             # 왼쪽 아래
        [int(width * 0.45) + offset_x, int(height * 0.6)],  # 왼쪽 위
        [int(width * 0.62) + offset_x, int(height * 0.6)],  # 오른쪽 위
        [int(width * 0.95) + offset_x, height]              # 오른쪽 아래
    ], np.int32)

    cv2.fillPoly(mask, [roi_points], 255)
    return mask, roi_points

frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)

    roi_mask, roi_points = create_lane_roi(frame)
    masked_edges = cv2.bitwise_and(edges, roi_mask)

    result_frame = frame.copy()
    cv2.polylines(result_frame, [roi_points], isClosed=True, color=(0, 255, 0), thickness=2)

    if frame_count % 10 == 0:
        combined = np.hstack((
            result_frame,
            cv2.cvtColor(masked_edges, cv2.COLOR_GRAY2BGR)
        ))
        cv2_imshow(combined)

    frame_count += 1
    if frame_count > 100:
        break

cap.release()
print("차선 검출 처리 완료!")